# Лабораторная работа 2
### Отбор комбинаций смежных слов со словом nearest, стоящим на 1 месте.
### Ранжирование комбинаций по частоте встречаемости.

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import scipy

In [18]:
# записываем текст в text
with open('text_sent_1.txt', 'r') as file:
    text = file.read().splitlines()

In [19]:
word_combinations = []
for sentence in text:
    words = sentence.split()
    if len(words) > 1:
        for i in range(len(words) - 1):
            word_combinations.append((words[i], words[i + 1]))

In [22]:
df = pd.DataFrame(word_combinations, columns=['first_word', 'second_word'])
df = df.groupby(['first_word', 'second_word']).size().reset_index(name='O11')
df

,first_word,second_word,O11
0,aa,ab,1
1,aa,aj,1
2,aa,and,4
3,aa,log,1
4,aa,the,1
...,...,...,...
67719,Вµr,where,1
67720,Вµt,procrustes,1
67721,Вµtk,Вµk,1
67722,Вµy,Вµz,1


In [25]:
# Подсчет частот встречаемости слов в других комбинациях
n = df['O11'].sum() 

first_word_counts = df.groupby('first_word')['O11'].sum().reset_index(name='total_first_word_count')
df = df.merge(first_word_counts, on='first_word', how='left')
df['O12'] = df['total_first_word_count'] - df['O11']

second_word_counts = df.groupby('second_word')['O11'].sum().reset_index(name='total_second_word_count')
df = df.merge(second_word_counts, on='second_word', how='left')
df['O21'] = df['total_second_word_count'] - df['O11']

df['O22'] = n - df['O11'] - df['O12'] - df['O21']
df

,first_word,second_word,O11,total_first_word_count,O12,total_second_word_count,O21,O22
0,aa,ab,1,8,7,2,1,176560
1,aa,aj,1,8,7,20,19,176542
2,aa,and,4,8,4,4769,4765,171796
3,aa,log,1,8,7,342,341,176220
4,aa,the,1,8,7,14047,14046,162515
...,...,...,...,...,...,...,...,...
67719,Вµr,where,1,4,3,436,435,176130
67720,Вµt,procrustes,1,1,0,17,16,176552
67721,Вµtk,Вµk,1,1,0,24,23,176545
67722,Вµy,Вµz,1,1,0,1,0,176568


In [26]:
# сортировка по частоте биграмм (O11)
df = df.sort_values(by=['O11'], ascending=False)

In [46]:
# дф только со словом nearest
df_nearest = df[(df['first_word'] == 'nearest') & (df['O11'] > 5)].sort_values(by=['O11'], ascending=False)
df_nearest = df_nearest.reset_index(drop=True)
df_nearest

,first_word,second_word,O11,total_first_word_count,O12,total_second_word_count,O21,O22
0,nearest,neighbor,133,266,133,137,4,176299
1,nearest,neighbors,82,266,184,105,23,176280
2,nearest,shrunken,22,266,244,34,12,176291
3,nearest,neighborhood,9,266,257,66,57,176246
4,nearest,centroid,7,266,259,32,25,176278


### Подсчет статистики Хи-2 Пирсона

In [55]:
df_nearest['chi2'] = 0 # Добавляем столбец для хи-квадрат

for index, row in df_nearest.iterrows():
    w1 = row['first_word']
    w2 = row['second_word']

    O11 = row['O11']
    O12 = row['O12']
    O21 = row['O21']
    O22 = row['O22']

    N = O11 + O12 + O21 + O22
    E11 = (O11 + O12) * (O11 + O21) / N
    E12 = (O11 + O12) * (O12 + O22) / N
    E21 = (O21 + O22) * (O11 + O21) / N
    E22 = (O21 + O22) * (O12 + O22) / N

    # Расчет хи-квадрат
    chi2 = ((O11 - E11)**2 / E11) + ((O12 - E12)**2 / E12) + ((O21 - E21)**2 / E21) + ((O22 - E22)**2 / E22)

    df_nearest.loc[index, 'chi2'] = chi2


C:\Users\Екатерина\AppData\Local\Temp\ipykernel_11420\2604188214.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '85636.4126768437' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_nearest.loc[index, 'chi2'] = chi2


In [57]:
df_nearest.sort_values(by='chi2', ascending=False)

,first_word,second_word,O11,total_first_word_count,O12,total_second_word_count,O21,O22,chi2
0,nearest,neighbor,133,266,133,137,4,176299,85636.412677
1,nearest,neighbors,82,266,184,105,23,176280,42433.333737
2,nearest,shrunken,22,266,244,34,12,176291,9421.347758
4,nearest,centroid,7,266,259,32,25,176278,1004.176095
3,nearest,neighborhood,9,266,257,66,57,176246,798.255258
